# Visualisation example Part 1

This notebook produces a GIF showing the progression of holdsAt() fluents which relate to a particular record over time. A prolog query 
finds all records which have been linked to an organ donation decision before a given time period, and the output from this file is stored in a file. The output is processed to produce a graph of known records at each time period, which are saved as PNGs and concatenated into a GIF using moviepy. 

NB: For unknown reasons, importing moviepy on the Prolog focused kernel is not working. This notebook contains the visualisation set up until moviepy is needed, at which point **VisualisationPart2.ipynb**, which was run in a Python kernel shows the rest of the example!

In [ ]:
% Set up required to reset environment
?- cd('~/work'), ['dec'].
?- initialiseDEC.
?- retractall(happensAtNarrative(_,_)).


true.
false.
true.

The cell below contains basic initiates statements from the wider organ donation example. Exp_rules have not been included here for simplicity. 

In order to easily identify fluents by IDs, events are represented in the form record(ID,event).

In this scenario the following interactions take place between the hospital and other medical sources:
- *hist_request(Patient, Source)* represents that a patient history request for *Patient* is sent to *Source*. For example, *hist_request(123,'Well')* represents that patient 123's history is requested from Wellington.
- *message(RID, M, Source)* represents that message *M* has been recieved from *Source* and is a response to history request *RID*. For example, *message(10, happensAt(diagnosis(123,'Hep-C','Positive'),1),'Well')* represents that a message has been received from Wellington, which is a response to request 10 and which provides information about a diagnosis event which occurred at time period 1. *message* initiates *messaged*.
- *hist_request* initiates *hist_requested*, which is a multi-valued fluent. *hist_requested='NA'* until a message has been received which answers the patient history request, and then it equals 'ANS'.
- A *testRequest(Patient,Test,Lab,Requester)* represents a request for a medical test, such as for HIV, to take place for a particular patient. This is sent to a lab. For example, *testRequest(123,'HIV','OtagoBlood','EH')* represents that the medical actor 'EH' has requested that the lab Otago Blood perform an HIV test on a sample from patient 123.
- A *resultMessage(RID,Res,Lab)* represents a message from a lab containing the result from a particular medical test. For example, *resultMessage(5,positive,'OtagoBlood') represents that the hospital has received a message from Otago Blood saying that the result for the test requested by record 5 is positive.
- A *testRequest(Patient,Test,Lab,Requester)* is set to 'NA' until a corresponding *resultMessage* is received, when it is set to 'ANS'.

Within the hospital, other events take place. Permissions for different medical staff has not been included here.
- A medical staff actor can declare a patient's brain death, represented by *brainDeath(PatID,Actor)*. *brainDeath* initiates *brainDead*.
- A doctor or other medical staff actor can diagnose a patient with a condition, represented by *diagnosis(Patient,Condition,Result,Actor)*. For example, *diagnosis(123,'HIV',pos,'EH')* represents a positive diagnosis of HIV for patient 123 by a medical actor with the ID 'EH'.
- After a patient's brain death, someone with appropriate clearance can decide that the patient is eligible to donate an organ. This is represented by *donorDecision(PatID,Concl,Actor)*, e.g. *donorDecision(123,yes,'EH')* represents that 'EH' has decided that patient 123 can be matched to a recipient for their organ. *donorDecision* initiates *donorDecided*.
- A diagnosis or donor decision can have evidence linked to it using *linkEvidence(ID,EID,Actor)*, where *ID* is the ID of the decision, *EID* is the ID of the evidence and *Actor* is the medical actor linking the evidence. *linkEvidence* initiated *linkedEvidence*




In [ ]:
% File: PatientCareProcedure.pl

% History requests are not yet fulfilled by default. History grants answer history requests
initiates(record(ID,histRequest(Patient,Source)), record(ID,histRequested(Patient,Source)='NA'), _).
initiates(record(_,message(ID, _, Source)), record(ID,histRequested(Patient,Source)='ANS'),T):- holdsAt(record(ID,histRequested(Patient,Source)='NA'),T).
initiates(record(EID,resultMessage(RID,Res,Lab)), record(EID,resultMessaged(RID,Res,Lab)),T).
% Result Messages answer test requests
initiates(record(ID,testRequest(Patient,Test,Lab,Requester)), record(ID,testRequested(Patient,Test,Lab,Requester)='NA'), _).
initiates(record(_,resultMessage(ID, _, Source)), record(ID,testRequested(_,_,Source,_)='ANS'),T):- holdsAt(record(ID,testRequested,_,_,Source,_)='NA',T).

% Event to fluent
initiates(record(ID,diagnosis(Patient,Condition,Result,Actor)), record(ID,diagnosed(Patient,Condition,Result,Actor)),_).
initiates(record(LID,linkEvidence(ID,EID,Actor)), record(LID,linkedEvidence(ID,EID,Actor)),_).
initiates(record(RID,donorDecision(PatID,Concl,Actor)), record(RID,donorDecided(PatID,Concl,Actor)),_).
initiates(record(NID,message(RID,M,Source)), record(NID,messaged(RID,M,Source)),_).
initiates(record(ID,brainDeath(PatID,Actor)), record(ID,brainDead(PatID,Actor)),_).


Below are the definitions for parent child relationships between records. For example, a message with a test result is the child of the request for that test, and a linkEvidence record connects a decision record to a evidence record.

 As state constraints have not yet been implemented, this code currently uses descendant/2 fluents rather than holdsAt(descendant/2) fluents. holdsAt() fluents should replace the existing fluents once dec.pl has been updated appropriately.

In [ ]:
% File: depends.pl
derived_fluent(descendant/2).
holdsAt(descendant(GID,RID),T):- holdsAt(child(GID,RID),T).
holdsAt(descendant(GID, RID),T) :- holdsAt(child(GID, MID),T), holdsAt(descendant(MID, RID),T).

initiates(record(RID,linkEvidence(DID,EID,Actor)), child(RID,DID),_).
initiates(record(RID,linkEvidence(DID,EID,Actor)), child(EID,RID),_).
initiates(record(NID,resultMessage(RID,_,_)), child(RID,NID),_).
initiates(record(NID,message(RID,_,_)), child(RID,NID),_).

Relevant narrative to model:
- At T = 1, Patient 124 is admitted to hospital (this event is not relevant to the others, and thus should not be visualised)
- At T = 2, 'AH' requests that Otago Blood perform an HIV test for patient 123.
- Also at T = 2, A history request for patient 123 is sent to Wellington.
- At T = 3, Otago Blood send a message back to the hospital saying that the test returned a positive result.
- At T = 4, 'PS' diagnoses patient 123 with HIV.
- Also at T = 4, The hospital receives a messagefrom Wellington with an historical record for patient 123 in response to the history request made at T = 2.
- At T = 5, 'PS' links the test result from Otago Blood received at T = 3 as evidence for the diagnosis made at T= 4.
- At T= 6, 'PS' declares Patient 123 to be brain dead (not included in visualisation)
- At T = 8, 'FG' decides that both patients 123 and 126 can be made donors. As Patient 126 is not the patient of interest, the donation decision made about them will not be included in the visualisation.
- At T = 9, 'FG' links the HIV diagnosis as evidence for their donor decision for patient 123, and also links the message received from Wellington Hospital about a diagnosis as evidence for the same decision.

In [ ]:
% File: narrative.pl

% Test narrative

happensAtNarrative(record(0,admission(124)),1).
happensAtNarrative(record(13,testRequest(123, 'HIV','Otago Blood', 'AH')),2).
happensAtNarrative(record(10,histRequest(123,'Well')),2).
happensAtNarrative(record(14,resultMessage(13,positive, 'Otago Blood')),3).
happensAtNarrative(record(11,diagnosis(123,'HIV',positive,'PS')),4).
happensAtNarrative(record(12,message(10, happensAt(diagnosis(123,'Hep-C','Positive'),1),'Well')),4).
happensAtNarrative(record(15,linkEvidence(11,14,'PS')),5).
happensAtNarrative(record(16,brainDeath(123,'PS')),6).
happensAtNarrative(record(18,donorDecision(123,yes,'FG')),8).
happensAtNarrative(record(19,donorDecision(126,yes,'FG')),8).
happensAtNarrative(record(20,linkEvidence(18,11,'FG')),9).
happensAtNarrative(record(21,linkEvidence(18,12,'FG')),9).

We are interested in visualising what is known between T = 0 and T = 12.

In [ ]:
?- run(12).

true.

In [ ]:
?- holdsAt(descendant(GID,RID),5).



GID = 13, RID = 14 ;
GID = 10, RID = 12 ;
GID = 13, RID = 14 ;
GID = 10, RID = 12 .

In [ ]:
?- holdsAt(child(GID,RID),X).

GID = 13, RID = 14, X = 4 ;
GID = 13, RID = 14, X = 5 ;
GID = 10, RID = 12, X = 5 ;
GID = 13, RID = 14, X = 6 ;
GID = 10, RID = 12, X = 6 ;
GID = 15, RID = 11, X = 6 ;
GID = 14, RID = 15, X = 6 ;
GID = 13, RID = 14, X = 7 ;
GID = 10, RID = 12, X = 7 ;
GID = 15, RID = 11, X = 7 ;
GID = 14, RID = 15, X = 7 ;
GID = 13, RID = 14, X = 8 ;
GID = 10, RID = 12, X = 8 ;
GID = 15, RID = 11, X = 8 ;
GID = 14, RID = 15, X = 8 ;
GID = 13, RID = 14, X = 9 ;
GID = 10, RID = 12, X = 9 ;
GID = 15, RID = 11, X = 9 ;
GID = 14, RID = 15, X = 9 ;
GID = 13, RID = 14, X = 10 ;
GID = 10, RID = 12, X = 10 ;
GID = 15, RID = 11, X = 10 ;
GID = 14, RID = 15, X = 10 ;
GID = 20, RID = 18, X = 10 ;
GID = 11, RID = 20, X = 10 .


findRelationships\3 finds all child parent relationships for records which are descendants of a specified record X, while findRecords\3 finds the records themselves.

In [ ]:
findRelationships(X,C,L):- findRelationships(X,C,L,X).
findRelationships(-1,_,[],_).
findRelationships(X,C,L,E):- X>= 0, Z is X - 1, findRelationships(Z, C,M,E), findall([X,Child,Parent],(holdsAt(child(Child,Parent),X), holdsAt(descendant(Child,C),E)), A), append(A,M,L2), sort(L2,L).
findRecords(X,C,L):- findRecords(X,C,L,X).
findRecords(-1,_,[],_).
findRecords(X,C,L):- X>= 0, Z is X - 1, findRecords(Z, C, M), findall([X,ID,Contents], (holdsAt(record(ID,Contents),X),(holdsAt(descendant(ID,C),E); ID = C)), A), append(A,M,L2), sort(L2,L).


We find the relationships and related records for record 18 at time 12 and save the output to a file.

In [ ]:
% Output: AnimationOutput.txt
% The query output is saved to the specified file
?- A = 12, findRelationships(A,18,X).
?- findRecords(12,18,X).

A = 12, X = [ [ 4, 13, 14 ], [ 5, 10, 12 ], [ 5, 13, 14 ], [ 6, 10, 12 ], [ 6, 13, 14 ], [ 6, 14, 15 ], [ 6, 15, 11 ], [ 7, 10, 12 ], [ 7, 13, 14 ], [ 7, 14, 15 ], [ 7, 15, 11 ], [ 8, 10, 12 ], [ 8, 13, 14 ], [ 8, 14, 15 ], [ 8, 15, 11 ], [ 9, 10, 12 ], [ 9, 13, 14 ], [ 9, 14, 15 ], [ 9, 15, 11 ], [ 10, 10, 12 ], [ 10, 11, 20 ], [ 10, 12, 21 ], [ 10, 13, 14 ], [ 10, 14, 15 ], [ 10, 15, 11 ], [ 10, 20, 18 ], [ 10, 21, 18 ], [ 11, 10, 12 ], [ 11, 11, 20 ], [ 11, 12, 21 ], [ 11, 13, 14 ], [ 11, 14, 15 ], [ 11, 15, 11 ], [ 11, 20, 18 ], [ 11, 21, 18 ], [ 12, 10, 12 ], [ 12, 11, 20 ], [ 12, 12, 21 ], [ 12, 13, 14 ], [ 12, 14, 15 ], [ 12, 15, 11 ], [ 12, 20, 18 ], [ 12, 21, 18 ] ] .
X = [ [ 3, 10, histRequested(123, Well)=NA ], [ 3, 13, testRequested(123, HIV, Otago Blood, AH)=NA ], [ 4, 10, histRequested(123, Well)=NA ], [ 4, 13, testRequested(123, HIV, Otago Blood, AH)=NA ], [ 4, 14, resultMessaged(13, positive, Otago Blood) ], [ 5, 10, histRequested(123, Well)=ANS ], [ 5, 10, histRequeste

We read the saved query output in from the file and save the relationships and records which hold at each time period from 0 up to the maximum (12) in allEdges and allNodes respectively.

In [ ]:
%Python
from array import *
# Read in prolog output
f = open("/work/output_files/AnimationOutput.txt", "r")
firstLine = f.readline()
# Remove extra spacing, brackets
relationships = firstLine[16:-7]
records = f.readline()[8:-7]
f.close()
# Save end time period
maxTime = int(firstLine.split(",")[0].split(" = ")[1])
# allEdges stores all the records that hold for each time period
allEdges = []
# allNodes stores all the parent-child relationships that hold for each time period
allNodes = []
for i in range(maxTime+1):
    allEdges.append({})
    allNodes.append({})
relationships = relationships.split(" ], [ ")
records = records.split(" ], [ ")
for rel in relationships:
    time, child, parent = rel.split(", ")
    time = int(time)
    if parent in allEdges[time]:
        allEdges[time][parent].append(child)
    else:
        allEdges[time][parent] = [child]
for rec in records:
    time = int(rec[0: rec.index(',')])
    rec = rec[rec.index(',')+1:]
    ID = rec[1: rec.index(',')]
    contents = rec[rec.index(',')+1:]
    allNodes[time][ID] = contents

In order to create a graph for each time period, we need to import pygraphviz. We add the edges and nodes to a new graph for each time period and save each graph as a PNG (it is also possible to produce SVGs, but there was less support for creating GIFs).

In [ ]:
%Python
import pygraphviz as pgv
from graphviz import Digraph
# Produce a PNG grpah for each time period
for i in range(maxTime+1):
    di = Digraph()
    di.graph_attr["label"] = "Time: %s" % i
    di.format = 'png'
    for key, value in allNodes[i].items():
        di.node(key,label=value)
    for key, value in allEdges[i].items():
        for v in value:
            di.edge(key,v)
    if i < 10:
        fname = "VisualisationExample/Timeline/di_0%s" % i
    else:
        fname = "VisualisationExample/Timeline/di_%s" % i
    di.render(filename=fname)

Converting the PNGs to a GIF uses moviepy. 

By default the PNG concatenation method centres PNGs of all sizes into the frame of the largest PNG, which makes the transition between images quite jolting. An alternative concatenate.py file could be used which centres PNGs horizontally but positions them at the bottom of the frame. (Unfortunately, Deepnote had issues with using git clone or other methods to install moviepy and change the contents of the file, so this example does not use the alternate file, although it did work at one point!)

In [ ]:
%Python
import os
os.system('pip install moviepy')
#os.system('cp /work/VisualisationExample/concatReplacement.py ~/venv/lib/python3.9/site-packages/moviepy/video/compositing/concatenate.py')


Here the cell (should!) concatenate the PNGs together to produce a GIF. It currently doesn't work in this kernel. See visualisationPart2.ipynb for the functional version.

In [ ]:
%Python
import glob
#from moviepy.editor import *
import moviepy.editor as mpv
input_png_list = glob.glob("/work/VisualisationExample/Timeline/*.png")
input_png_list.sort()
clips = [mpv.ImageClip(i).set_duration(2) for i in input_png_list]
for c in clips:
    print(c)
concat_clip = mpv.concatenate_videoclips(clips, method="compose", bg_color=[255,255,255])
concat_clip.write_gif("/work/VisualisationExample/process.gif", fps=2)

ERROR: Script gave error name 'mpv' is not defined

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=1527cc64-36a2-4b35-bd8b-8d493ca554fa' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>